In [258]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


INSTALL NECESSARY LIBRARY 

In [259]:
import lightgbm as lgb
import kaggle
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb
import warnings

EXTRACT FILE PATH

In [260]:
!kaggle datasets download -d fedesoriano/company-bankruptcy-prediction

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction
License(s): copyright-authors
company-bankruptcy-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [261]:
# Replace 'company-bankruptcy-prediction.zip' with the actual downloaded filename (if different)
zip_file = 'company-bankruptcy-prediction.zip'
destination_folder = 'unzipped_data'  # You can customize the folder name

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print('Dataset unzipped successfully!')

Dataset unzipped successfully!


READ DATASET FROM DIRECTORY

In [262]:
df = pd.read_csv('unzipped_data/data.csv')
df

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


In [263]:
df.columns

Index(['Bankrupt?', ' ROA(C) before interest and depreciation before interest',
       ' ROA(A) before interest and % after tax',
       ' ROA(B) before interest and depreciation after tax',
       ' Operating Gross Margin', ' Realized Sales Gross Margin',
       ' Operating Profit Rate', ' Pre-tax net Interest Rate',
       ' After-tax net Interest Rate',
       ' Non-industry income and expenditure/revenue',
       ' Continuous interest rate (after tax)', ' Operating Expense Rate',
       ' Research and development expense rate', ' Cash flow rate',
       ' Interest-bearing debt interest rate', ' Tax rate (A)',
       ' Net Value Per Share (B)', ' Net Value Per Share (A)',
       ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons',
       ' Cash Flow Per Share', ' Revenue Per Share (Yuan ¥)',
       ' Operating Profit Per Share (Yuan ¥)',
       ' Per Share Net profit before tax (Yuan ¥)',
       ' Realized Sales Gross Profit Growth Rate',
       ' Operating Profit

In [264]:
string_columns = []
float_columns = []
# Use items() instead of iteritems() for pandas versions >= 1.0.0
for col_name, col_type in df.dtypes.items():
    if col_type == np.str_:
        string_columns.append(col_name)
    elif col_type == np.float64:
        float_columns.append(col_name)

print("String Columns:", string_columns)
print("Float Columns:", float_columns)

String Columns: []
Float Columns: [' ROA(C) before interest and depreciation before interest', ' ROA(A) before interest and % after tax', ' ROA(B) before interest and depreciation after tax', ' Operating Gross Margin', ' Realized Sales Gross Margin', ' Operating Profit Rate', ' Pre-tax net Interest Rate', ' After-tax net Interest Rate', ' Non-industry income and expenditure/revenue', ' Continuous interest rate (after tax)', ' Operating Expense Rate', ' Research and development expense rate', ' Cash flow rate', ' Interest-bearing debt interest rate', ' Tax rate (A)', ' Net Value Per Share (B)', ' Net Value Per Share (A)', ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons', ' Cash Flow Per Share', ' Revenue Per Share (Yuan ¥)', ' Operating Profit Per Share (Yuan ¥)', ' Per Share Net profit before tax (Yuan ¥)', ' Realized Sales Gross Profit Growth Rate', ' Operating Profit Growth Rate', ' After-tax Net Profit Growth Rate', ' Regular Net Profit Growth Rate', ' Continuo

In [265]:
df.isnull().mean()

Bankrupt?                                                   0.0
 ROA(C) before interest and depreciation before interest    0.0
 ROA(A) before interest and % after tax                     0.0
 ROA(B) before interest and depreciation after tax          0.0
 Operating Gross Margin                                     0.0
                                                           ... 
 Liability to Equity                                        0.0
 Degree of Financial Leverage (DFL)                         0.0
 Interest Coverage Ratio (Interest expense to EBIT)         0.0
 Net Income Flag                                            0.0
 Equity to Liability                                        0.0
Length: 96, dtype: float64

DATA EXPLORATORY

In [266]:
features = list(df.columns)
features.remove('Bankrupt?')

In [267]:
for feature in features:
    df[feature] = df[feature].round(3)

In [268]:
num_rows = 32
num_cols = 3

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 300))
axes = axes.flatten()
for i, feature in enumerate(features):
    ax = axes[i]

    bankrupt_data = df[df['Bankrupt?'] == 1][feature]
    non_bankrupt_data = df[df['Bankrupt?'] == 0][feature]

    ax.boxplot([bankrupt_data, non_bankrupt_data], labels=['Bankrupt(1)', 'Non-Bankrupt(0)'], notch=True)
    ax.set_xlabel(feature)
    ax.set_ylabel("Distribution")
    ax.set_title("Boxplot of " + feature)
    ax.legend()

plt.tight_layout()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

In [269]:
df_class_0 = df[df['Bankrupt?'] == 0]
df_class_1 = df[df['Bankrupt?'] == 1]

In [270]:
print('Count rows of Bankrup == 1 :',len(df_class_1))
print('Count rows of Bankrup == 0 :',len(df_class_0))

Count rows of Bankrup == 1 : 220
Count rows of Bankrup == 0 : 6599


In [271]:
def count_outliers(df, threshold=1.5):
  for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):  # Check for numeric columns
      q1 = df[col].quantile(0.25)
      q3 = df[col].quantile(0.75)
      iqr = q3 - q1
      lower_bound = q1 - (iqr * threshold)
      upper_bound = q3 + (iqr * threshold)
      outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
  return outlier_count


In [272]:
num_outliers_class1 = count_outliers(df_class_1)
num_outliers_class0 = count_outliers(df_class_0)
print("Number of rows with outliers:", num_outliers_class1)
print("Number of rows with outliers:", num_outliers_class0)

Number of rows with outliers: 17
Number of rows with outliers: 521


In [273]:
def drop_outlier_rows(df):
  outlier_rows = []  # To store indices of rows to drop
  # Iterate through rows
  for index, row in df.iterrows():
    # Check for outliers in each column
    for col in df.columns:
      if pd.api.types.is_numeric_dtype(df[col]):  # Check for numeric columns
        # Calculate IQR-based outlier bounds
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * 1.5)  # Adjust threshold as needed
        upper_bound = q3 + (iqr * 1.5)

        # Check if current value is an outlier
        if row[col] < lower_bound or row[col] > upper_bound:
          outlier_rows.append(index)  # Add index to drop list
          break  # No need to check other columns for this row

  # Drop outlier rows (if any)
  if outlier_rows:
    df_filtered = df.drop(outlier_rows)
    return df_filtered
  else:
    return df  # No outliers found, return original DataFrame

In [274]:
df_class0_filtered = drop_outlier_rows(df_class_0)  # Use a copy to avoid modifying original data
print("DataFrame after outlier removal:")

DataFrame after outlier removal:


In [275]:
df_class0_filtered

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
8,0,0.489,0.545,0.543,0.604,0.604,0.999,0.797,0.809,0.304,...,0.804,0.001,0.624,0.604,0.840,0.276,0.027,0.566,1,0.066
17,0,0.536,0.590,0.581,0.618,0.618,0.999,0.798,0.809,0.304,...,0.827,0.003,0.624,0.618,0.842,0.277,0.027,0.566,1,0.049
21,0,0.505,0.564,0.553,0.607,0.607,0.999,0.797,0.809,0.304,...,0.814,0.001,0.624,0.607,0.841,0.278,0.027,0.566,1,0.038
42,0,0.511,0.567,0.559,0.618,0.618,0.999,0.798,0.809,0.304,...,0.814,0.003,0.624,0.618,0.841,0.278,0.027,0.566,1,0.044
46,0,0.502,0.564,0.546,0.609,0.609,0.999,0.797,0.809,0.303,...,0.814,0.001,0.624,0.609,0.841,0.278,0.027,0.565,1,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6658,0,0.521,0.576,0.567,0.614,0.614,0.999,0.798,0.809,0.304,...,0.821,0.001,0.624,0.614,0.842,0.277,0.027,0.565,1,0.057
6667,0,0.511,0.574,0.558,0.607,0.607,0.999,0.797,0.809,0.303,...,0.820,0.001,0.624,0.607,0.842,0.279,0.027,0.565,1,0.034
6742,0,0.483,0.541,0.534,0.616,0.616,0.999,0.797,0.809,0.303,...,0.802,0.001,0.624,0.616,0.840,0.277,0.027,0.566,1,0.048
6799,0,0.532,0.590,0.578,0.606,0.606,0.999,0.798,0.809,0.304,...,0.825,0.001,0.624,0.606,0.842,0.279,0.027,0.565,1,0.034


In [276]:
less_information_feature = []

In [277]:
for feature in features:
    if len(df_class0_filtered[feature].unique()) <10:
        less_information_feature.append(feature)

In [278]:
df_class0_filtered.drop(columns = less_information_feature,inplace=True)
df_class_1.drop(columns = less_information_feature,inplace=True)

C:\Users\MTL80204\AppData\Local\Temp\ipykernel_4932\537461874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class_1.drop(columns = less_information_feature,inplace=True)


In [279]:
df_class0_filtered.rename(columns={'Bankrupt?': 'Bankrup'}, inplace=True)
df_class_1.rename(columns={'Bankrupt?': 'Bankrup'}, inplace=True)

C:\Users\MTL80204\AppData\Local\Temp\ipykernel_4932\2406957564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class_1.rename(columns={'Bankrupt?': 'Bankrup'}, inplace=True)


In [280]:
df_appended = pd.concat([df_class0_filtered, df_class_1], ignore_index=True)

In [281]:
scaler = MinMaxScaler(feature_range=(0, 1))  # Set range to (0, 1)
df_scaled = pd.DataFrame(scaler.fit_transform(df_appended), columns=df_appended.columns)

In [282]:
error_col = []
for col in df_appended.columns:
    if df_appended[col].mean()>1:
        error_col.append(col)

In [283]:
df_appended.drop(columns= error_col,inplace=True)
df_appended

,Bankrup,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Cash flow rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),...,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Net Income to Total Assets,Gross Profit to Sales,Liability to Equity,Equity to Liability
0,0,0.489,0.545,0.543,0.604,0.604,0.463,0.006,0.184,0.184,...,0.111,0.650,0.462,0.574,0.315,0.022,0.804,0.604,0.276,0.066
1,0,0.536,0.590,0.581,0.618,0.618,0.470,0.217,0.190,0.190,...,0.116,0.640,0.458,0.601,0.314,0.018,0.827,0.618,0.277,0.049
2,0,0.505,0.564,0.553,0.607,0.607,0.471,0.209,0.185,0.185,...,0.111,0.683,0.469,0.623,0.320,0.019,0.814,0.607,0.278,0.038
3,0,0.511,0.567,0.559,0.618,0.618,0.467,0.236,0.187,0.187,...,0.115,0.660,0.464,0.595,0.317,0.025,0.814,0.618,0.278,0.044
4,0,0.502,0.564,0.546,0.609,0.609,0.469,0.296,0.185,0.185,...,0.111,0.643,0.459,0.611,0.315,0.018,0.814,0.609,0.278,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,1,0.419,0.434,0.461,0.613,0.613,0.460,0.000,0.139,0.139,...,0.111,0.634,0.458,0.535,0.311,0.045,0.726,0.613,0.293,0.016
511,1,0.197,0.211,0.221,0.598,0.598,0.473,0.000,0.070,0.070,...,0.090,0.642,0.459,0.739,0.315,0.059,0.519,0.598,0.259,0.004
512,1,0.338,0.254,0.378,0.591,0.591,0.465,0.000,0.131,0.131,...,0.145,0.609,0.456,0.650,0.282,0.051,0.558,0.591,0.337,0.012
513,1,0.340,0.345,0.380,0.581,0.581,0.458,0.000,0.132,0.132,...,0.217,0.649,0.460,0.511,0.320,0.061,0.642,0.581,0.337,0.012


In [284]:
for col in df_appended.columns:
    print('column name :',col+' (mean) =',df_appended[col].mean())

column name : Bankrup (mean) = 0.42718446601941745
column name :  ROA(C) before interest and depreciation before interest (mean) = 0.47155728155339804
column name :  ROA(A) before interest and % after tax (mean) = 0.5205728155339806
column name :  ROA(B) before interest and depreciation after tax (mean) = 0.5178640776699031
column name :  Operating Gross Margin (mean) = 0.6047223300970873
column name :  Realized Sales Gross Margin (mean) = 0.6047223300970874
column name :  Cash flow rate (mean) = 0.46389902912621356
column name :  Tax rate (A) (mean) = 0.10852815533980582
column name :  Net Value Per Share (B) (mean) = 0.1764718446601942
column name :  Net Value Per Share (A) (mean) = 0.17643106796116503
column name :  Net Value Per Share (C) (mean) = 0.17651650485436896
column name :  Persistent EPS in the Last Four Seasons (mean) = 0.21165436893203884
column name :  Cash Flow Per Share (mean) = 0.32049514563106796
column name :  Revenue Per Share (Yuan ¥) (mean) = 0.03227572815533981

In [285]:
df_test = df_appended.copy()

FEATURE IMPORTANCE

In [286]:
y = df_test['Bankrup']
X = df_test.drop('Bankrup', axis=1)

In [287]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [288]:
model = RandomForestClassifier().fit(X_train, y_train)
feature_importances = model.feature_importances_

In [289]:
feature_names = X.columns  # Replace with your column names if obtained differently

# Combine feature names and importances into a DataFrame
importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})

# Sort DataFrame by importance (descending order)
importance_df = importance_df.sort_values(by='importance', ascending=False)
importance_df

,feature,importance
36,Retained Earnings to Total Assets,0.132822
10,Persistent EPS in the Last Four Seasons,0.132491
46,Net Income to Total Assets,0.086016
0,ROA(C) before interest and depreciation befor...,0.074214
21,Net profit before tax/Paid-in capital,0.062122
2,ROA(B) before interest and depreciation after...,0.056244
30,Quick Assets/Current Liability,0.050088
1,ROA(A) before interest and % after tax,0.047366
19,Borrowing dependency,0.042546
14,Per Share Net profit before tax (Yuan ¥),0.039080


In [290]:
threshold = 0.01
df_selected = importance_df[importance_df['importance'] >= threshold]
# Print or use the filtered DataFrame
print("Filtered DataFrame:")
print(df_selected)

Filtered DataFrame:
                                              feature  importance
36                  Retained Earnings to Total Assets    0.132822
10            Persistent EPS in the Last Four Seasons    0.132491
46                         Net Income to Total Assets    0.086016
0    ROA(C) before interest and depreciation befor...    0.074214
21              Net profit before tax/Paid-in capital    0.062122
2    ROA(B) before interest and depreciation after...    0.056244
30                     Quick Assets/Current Liability    0.050088
1              ROA(A) before interest and % after tax    0.047366
19                               Borrowing dependency    0.042546
14           Per Share Net profit before tax (Yuan ¥)    0.039080
6                                        Tax rate (A)    0.029782
49                                Equity to Liability    0.028206
18                                   Net worth/Assets    0.021836
9                             Net Value Per Share (C)   

In [291]:
list_col_use = list(df_selected['feature'].unique())
list_col_use.append('Bankrup')

In [292]:
columns_to_drop = [col for col in df_test.columns if col not in list_col_use]
completed_select = df_test.drop(columns_to_drop, axis=1)

In [293]:
completed_select

,Bankrup,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Tax rate (A),Net Value Per Share (B),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Per Share Net profit before tax (Yuan ¥),Current Ratio,...,Borrowing dependency,Net profit before tax/Paid-in capital,Operating profit per person,Working Capital to Total Assets,Quick Assets/Current Liability,Retained Earnings to Total Assets,Current Liability to Current Assets,Net Income to Total Assets,Liability to Equity,Equity to Liability
0,0,0.489,0.545,0.543,0.006,0.184,0.184,0.218,0.173,0.013,...,0.370,0.172,0.392,0.814,0.012,0.935,0.022,0.804,0.276,0.066
1,0,0.536,0.590,0.581,0.217,0.190,0.190,0.233,0.189,0.016,...,0.373,0.188,0.401,0.816,0.010,0.949,0.018,0.827,0.277,0.049
2,0,0.505,0.564,0.553,0.209,0.185,0.185,0.225,0.182,0.016,...,0.371,0.181,0.398,0.847,0.013,0.938,0.019,0.814,0.278,0.038
3,0,0.511,0.567,0.559,0.236,0.187,0.187,0.225,0.181,0.012,...,0.373,0.180,0.397,0.801,0.007,0.945,0.025,0.814,0.278,0.044
4,0,0.502,0.564,0.546,0.296,0.185,0.185,0.223,0.183,0.017,...,0.371,0.182,0.401,0.855,0.015,0.938,0.018,0.814,0.278,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,1,0.419,0.434,0.461,0.000,0.139,0.139,0.198,0.150,0.006,...,0.390,0.148,0.389,0.765,0.006,0.874,0.045,0.726,0.293,0.016
511,1,0.197,0.211,0.221,0.000,0.070,0.070,0.080,0.054,0.005,...,0.357,0.057,0.386,0.723,0.004,0.778,0.059,0.519,0.259,0.004
512,1,0.338,0.254,0.378,0.000,0.131,0.131,0.165,0.092,0.006,...,0.403,0.091,0.379,0.740,0.003,0.853,0.051,0.558,0.337,0.012
513,1,0.340,0.345,0.380,0.000,0.132,0.132,0.154,0.106,0.005,...,0.459,0.106,0.361,0.720,0.002,0.879,0.061,0.642,0.337,0.012


MODELING

In [294]:
y = completed_select['Bankrup']
X = completed_select.drop('Bankrup', axis=1)

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [296]:
model = RandomForestClassifier(random_state=42)

In [297]:
params= {   
    "n_estimators":range(100, 200 , 300),
    "max_depth": range(10 , 20 , 30)
}
model = GridSearchCV(
    model,
    param_grid= params,
    cv=5,
    n_jobs=-1,
    verbose= 1
)
model.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(10, 20, 30),
                         'n_estimators': range(100, 200, 300)},
             verbose=1)

In [298]:
model.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [299]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9611650485436893


LightGBM

In [300]:
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 181, number of negative: 231
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1439
[LightGBM] [Info] Number of data points in the train set: 412, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.439320 -> initscore=-0.243921
[LightGBM] [Info] Start training from score -0.243921
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

LGBMClassifier()

In [301]:
y_pred=clf.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9612


In [302]:
#LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               #importance_type='split', learning_rate=0.1, max_depth=-1,
               #min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               #n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               #random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               #subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
params = {
    "boosting_type": ["gbdt", "rf"],  # Convert range to a list of actual values
    "learning_rate": [0.01,0.005,0.001],  # Convert range to list of values
    "max_depth": list(range(10, 20, 5)),  # Convert range to list of values
    "min_data_in_leaf": list(range(20, 30, 4)),  # Convert range to list of values
    "n_estimators": list(range(100, 150, 200)) + [300],  # Convert range and add 300
}
model = GridSearchCV(
    clf,
    param_grid= params,
    cv=10,
    n_jobs=-1,
    verbose= 1
)
#model.fit(X_train, y_train)

Fitting 10 folds for each of 72 candidates, totalling 720 fits


c:\Users\MTL80204\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
360 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\MTL80204\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\MTL80204\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\sklearn.py", line 1187, in fit
    super().fit(
  File "c:\Users\MTL80204\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\sklearn

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 181, number of negative: 231
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000186 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1439
[LightGBM] [Info] Number of data points in the train set: 412, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.439320 -> in

GridSearchCV(cv=10, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt', 'rf'],
                         'learning_rate': [0.01, 0.005, 0.001],
                         'max_depth': [10, 15],
                         'min_data_in_leaf': [20, 24, 28],
                         'n_estimators': [100, 300]},
             verbose=1)

In [303]:
#model.best_params_

{'boosting_type': 'gbdt',
 'learning_rate': 0.01,
 'max_depth': 10,
 'min_data_in_leaf': 20,
 'n_estimators': 300}